In [2]:
import os
import sys
import h5py
import json
import numpy as np
import torch as pt
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob

from src.dataset import StructuresDataset, collate_batch_features, select_by_sid, select_by_interface_types
from src.data_encoding import encode_structure, encode_features, extract_topology, categ_to_resnames, resname_to_categ
from src.structure import data_to_structure, encode_bfactor, concatenate_chains, split_by_chain
from src.structure_io import save_pdb, read_pdb
from src.scoring import bc_scoring, bc_score_names

In [3]:
# data parameters
data_path = "examples/issue_20_04_2023"

In [4]:
# model parameters
# R3
#save_path = "model/save/i_v3_0_2021-05-27_14-27"  # 89
#save_path = "model/save/i_v3_1_2021-05-28_12-40"  # 90
# R4
#save_path = "model/save/i_v4_0_2021-09-07_11-20"  # 89
save_path = "model/save/i_v4_1_2021-09-07_11-21"  # 91

# select saved model
model_filepath = os.path.join(save_path, 'model_ckpt.pt')
#model_filepath = os.path.join(save_path, 'model.pt')

In [5]:
# add module to path
if save_path not in sys.path:
    sys.path.insert(0, save_path)
    
# load functions
from config import config_model, config_data
from data_handler import Dataset
from model import Model

In [6]:
# define device
device = pt.device("cpu")

# create model
model = Model(config_model)

# reload model
model.load_state_dict(pt.load(model_filepath, map_location=pt.device("cpu")))

# set model to inference
model = model.eval().to(device)

In [7]:
# find pdb files and ignore already predicted oins
pdb_filepaths = glob(os.path.join('Test18PDB1', "*.pdb1"), recursive=True)
pdb_filepaths = [fp for fp in pdb_filepaths if "_i" not in fp]
# pdb_filepaths = ['1uud.pdb']

# create dataset loader with preprocessing
dataset = StructuresDataset(pdb_filepaths, with_preprocessing=True)

# debug print
print(len(dataset))

18


In [8]:
# run model on all subunits
with pt.no_grad():
    for subunits, filepath in tqdm(dataset):
        # concatenate all chains together
        structure = concatenate_chains(subunits)

        # encode structure and features
        X, M = encode_structure(structure)
        #q = pt.cat(encode_features(structure), dim=1)
        q = encode_features(structure)[0]

        # extract topology
        ids_topk, _, _, _, _ = extract_topology(X, 64)

        # pack data and setup sink (IMPORTANT)
        X, ids_topk, q, M = collate_batch_features([[X, ids_topk, q, M]])

        # run model
        z = model(X.to(device), ids_topk.to(device), q.to(device), M.float().to(device))

        # for all predictions
        for i in range(z.shape[1]):
            # prediction
            p = pt.sigmoid(z[:,i])
            # encode result
            structure = encode_bfactor(structure, p.cpu().numpy())

            # save results
            output_filepath = 'Test18predictions/' + filepath[:-4]+'_i{}.pdb'.format(i)
            save_pdb(split_by_chain(structure), output_filepath)

  0%|          | 0/18 [00:00<?, ?it/s]

Test18PDB1/379d.pdb1


/Users/riddhishthakare/Downloads/PeSTo/penv/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
  6%|▌         | 1/18 [00:11<03:09, 11.14s/it]

Test18PDB1/1fmn.pdb1


 11%|█         | 2/18 [00:21<02:49, 10.61s/it]

...

 89%|████████▉ | 16/18 [03:25<00:22, 11.13s/it]

Test18PDB1/4yaz.pdb1


 94%|█████████▍| 17/18 [03:45<00:13, 13.80s/it]

Test18PDB1/5bjo.pdb1


100%|██████████| 18/18 [04:04<00:00, 13.58s/it]
